In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Mortgage Forecasting - Data Wrangling\n",
    "## HMDA Data Processing and Aggregation"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from pathlib import Path\n",
    "import yaml\n",
    "import warnings\n",
    "warnings.filterwarnings('ignore')\n",
    "\n",
    "# Set up plotting\n",
    "plt.style.use('seaborn-v0_8')\n",
    "sns.set_palette(\"husl\")\n",
    "%matplotlib inline"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Configuration and Setup"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Load configuration\n",
    "with open('../config/config.yaml', 'r') as file:\n",
    "    config = yaml.safe_load(file)\n",
    "\n",
    "print(\"Project Configuration:\")\n",
    "print(f\"Target Geography: {config['data']['target_geography']}\")\n",
    "print(f\"Geography Code: {config['data']['geography_code']}\")\n",
    "print(f\"Years: {config['data']['hmda_years']}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. HMDA Data Loading Function"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def load_hmda_year(year):\n",
    "    \"\"\"Load HMDA data for a specific year\"\"\"\n",
    "    file_path = Path(f\"../data/raw/hmda_{year}.csv\")\n",
    "    \n",
    "    if not file_path.exists():\n",
    "        print(f\"File not found: {file_path}\")\n",
    "        return None\n",
    "    \n",
    "    print(f\"Loading HMDA {year}...\")\n",
    "    \n",
    "    # Read with optimized dtypes\n",
    "    dtype_dict = {\n",
    "        'loan_amount': 'float64',\n",
    "        'action_taken': 'int64',\n",
    "        'state_code': 'str',\n",
    "        'county_code': 'str',\n",
    "        'msa_md': 'str',\n",
    "        'applicant_income_000s': 'float64',\n",
    "        'as_of_year': 'int64'\n",
    "    }\n",
    "    \n",
    "    try:\n",
    "        df = pd.read_csv(file_path, dtype=dtype_dict, low_memory=False)\n",
    "        print(f\"  Loaded {len(df):,} records\")\n",
    "        return df\n",
    "    except Exception as e:\n",
    "        print(f\"  Error loading {year}: {e}\")\n",
    "        return None\n",
    "\n",
    "# Load all years\n",
    "all_data = []\n",
    "for year in config['data']['hmda_years']:\n",
    "    df_year = load_hmda_year(year)\n",
    "    if df_year is not None:\n",
    "        all_data.append(df_year)\n",
    "\n",
    "if all_data:\n",
    "    hmda_raw = pd.concat(all_data, ignore_index=True)\n",
    "    print(f\"\\nTotal combined records: {len(hmda_raw):,}\")\n",
    "else:\n",
    "    print(\"No data loaded!\")\n",
    "    hmda_raw = pd.DataFrame()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Data Exploration and Understanding"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if not hmda_raw.empty:\n",
    "    print(\"Data Overview:\")\n",
    "    print(f\"Shape: {hmda_raw.shape}\")\n",
    "    print(\"\\nColumns:\")\n",
    "    print(hmda_raw.columns.tolist())\n",
    "    \n",
    "    print(\"\\nSample data:\")\n",
    "    display(hmda_raw.head())\n",
    "    \n",
    "    print(\"\\nData types:\")\n",
    "    display(hmda_raw.dtypes)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Check action_taken distribution\n",
    "if 'action_taken' in hmda_raw.columns:\n",
    "    print(\"Action Taken Distribution:\")\n",
    "    action_counts = hmda_raw['action_taken'].value_counts().sort_index()\n",
    "    \n",
    "    # Common HMDA action codes\n",
    "    action_map = {\n",
    "        1: 'Loan originated',\n",
    "        2: 'Application approved but not accepted',\n",
    "        3: 'Application denied',\n",
    "        4: 'Application withdrawn by applicant',\n",
    "        5: 'File closed for incompleteness',\n",
    "        6: 'Purchased loan',\n",
    "        7: 'Preapproval request denied',\n",
    "        8: 'Preapproval request approved but not accepted'\n",
    "    }\n",
    "    \n",
    "    for code, count in action_counts.items():\n",
    "        desc = action_map.get(code, 'Unknown')\n",
    "        print(f\"  {code}: {desc} - {count:,} records ({count/len(hmda_raw)*100:.1f}%)\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Data Filtering and Cleaning"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def filter_originated_loans(df):\n",
    "    \"\"\"Filter for originated loans only (action_taken = 1)\"\"\"\n",
    "    if 'action_taken' not in df.columns:\n",
    "        print(\"Warning: 'action_taken' column not found\")\n",
    "        return df\n",
    "    \n",
    "    original_count = len(df)\n",
    "    df_originated = df[df['action_taken'] == 1].copy()\n",
    "    \n",
    "    print(f\"Filtered to originated loans: {len(df_originated):,} records ({len(df_originated)/original_count*100:.1f}% of total)\")\n",
    "    \n",
    "    return df_originated\n",
    "\n",
    "def filter_target_geography(df, geography_type, geography_code):\n",
    "    \"\"\"Filter for target geography\"\"\"\n",
    "    if geography_type == 'msa':\n",
    "        if 'msa_md' not in df.columns:\n",
    "            print(\"Warning: 'msa_md' column not found\")\n",
    "            return df\n",
    "        df_geo = df[df['msa_md'] == geography_code].copy()\n",
    "        print(f\"Filtered to MSA {geography_code}: {len(df_geo):,} records\")\n",
    "    else:  # state\n",
    "        if 'state_code' not in df.columns:\n",
    "            print(\"Warning: 'state_code' column not found\")\n",
    "            return df\n",
    "        df_geo = df[df['state_code'] == geography_code].copy()\n",
    "        print(f\"Filtered to state {geography_code}: {len(df_geo):,} records\")\n",
    "    \n",
    "    return df_geo\n",
    "\n",
    "# Apply filters\n",
    "if not hmda_raw.empty:\n",
    "    print(\"Applying data filters...\")\n",
    "    \n",
    "    # Filter for originated loans\n",
    "    hmda_originated = filter_originated_loans(hmda_raw)\n",
    "    \n",
    "    # Filter for target geography\n",
    "    hmda_filtered = filter_target_geography(\n",
    "        hmda_originated, \n",
    "        config['data']['geography_type'], \n",
    "        config['data']['geography_code']\n",
    "    )\n",
    "    \n",
    "    print(f\"\\nFinal filtered dataset: {len(hmda_filtered):,} records\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Data Quality Checks"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if not hmda_filtered.empty:\n",
    "    print(\"Data Quality Checks:\")\n",
    "    \n",
    "    # Check for missing values\n",
    "    print(\"\\nMissing Values:\")\n",
    "    missing_data = hmda_filtered.isnull().sum()\n",
    "    for col, missing_count in missing_data.items():\n",
    "        if missing_count > 0:\n",
    "            print(f\"  {col}: {missing_count:,} missing ({missing_count/len(hmda_filtered)*100:.1f}%)\")\n",
    "    \n",
    "    # Check loan amount distribution\n",
    "    if 'loan_amount' in hmda_filtered.columns:\n",
    "        print(f\"\\nLoan Amount Statistics:\")\n",
    "        print(f\"  Min: ${hmda_filtered['loan_amount'].min():,.0f}\")\n",
    "        print(f\"  Mean: ${hmda_filtered['loan_amount'].mean():,.0f}\")\n",
    "        print(f\"  Median: ${hmda_filtered['loan_amount'].median():,.0f}\")\n",
    "        print(f\"  Max: ${hmda_filtered['loan_amount'].max():,.0f}\")\n",
    "        \n",
    "        # Check for zeros or negative values\n",
    "        zero_or_negative = (hmda_filtered['loan_amount'] <= 0).sum()\n",
    "        if zero_or_negative > 0:\n",
    "            print(f\"  WARNING: {zero_or_negative} records with zero or negative loan amounts\")\n",
    "    \n",
    "    # Check temporal coverage\n",
    "    if 'as_of_year' in hmda_filtered.columns:\n",
    "        print(f\"\\nYearly Distribution:\")\n",
    "        year_counts = hmda_filtered['as_of_year'].value_counts().sort_index()\n",
    "        for year, count in year_counts.items():\n",
    "            print(f\"  {year}: {count:,} records\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Time Series Aggregation"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def create_quarterly_series(df):\n",
    "    \"\"\"Aggregate loan volume by quarter\"\"\"\n",
    "    if df.empty:\n",
    "        return pd.DataFrame()\n",
    "    \n",
    "    # Create quarter dates (approximate - HMDA doesn't have exact dates)\n",
    "    # We'll assign each record to a quarter based on year\n",
    "    df_agg = df.copy()\n",
    "    \n",
    "    # Create quarter mapping\n",
    "    quarter_dates = []\n",
    "    for year in df_agg['as_of_year'].unique():\n",
    "        for quarter in [1, 2, 3, 4]:\n",
    "            if quarter == 1:\n",
    "                date = f\"{year}-03-31\"\n",
    "            elif quarter == 2:\n",
    "                date = f\"{year}-06-30\"\n",
    "            elif quarter == 3:\n",
    "                date = f\"{year}-09-30\"\n",
    "            else:\n",
    "                date = f\"{year}-12-31\"\n",
    "            quarter_dates.append(date)\n",
    "    \n",
    "    # For simplicity, we'll assign records randomly to quarters within their year\n",
    "    # In practice, you might have actual application dates\n",
    "    np.random.seed(42)  # for reproducibility\n",
    "    df_agg['quarter_date'] = pd.to_datetime(\n",
    "        df_agg['as_of_year'].astype(str) + \n",
    "        '-Q' + \n",
    "        pd.Series(np.random.choice([1, 2, 3, 4], len(df_agg))).astype(str)\n",
    "    ).dt.to_period('Q').dt.end_time\n",
    "    \n",
    "    # Aggregate by quarter\n",
    "    quarterly = df_agg.groupby('quarter_date')['loan_amount'].agg([\n",
    "        ('total_loan_volume', 'sum'),\n",
    "        ('loan_count', 'count'),\n",
    "        ('avg_loan_size', 'mean')\n",
    "    ]).reset_index()\n",
    "    \n",
    "    quarterly = quarterly.rename(columns={'quarter_date': 'date'})\n",
    "    quarterly = quarterly.sort_values('date')\n",
    "    \n",
    "    # Create complete time series\n",
    "    start_date = quarterly['date'].min()\n",
    "    end_date = quarterly['date'].max()\n",
    "    all_quarters = pd.date_range(start=start_date, end=end_date, freq='Q')\n",
    "    \n",
    "    complete_series = pd.DataFrame({'date': all_quarters})\n",
    "    complete_series = complete_series.merge(quarterly, on='date', how='left')\n",
    "    \n",
    "    # Fill missing values with 0 (assuming no loans in that quarter)\n",
    "    complete_series['total_loan_volume'] = complete_series['total_loan_volume'].fillna(0)\n",
    "    complete_series['loan_count'] = complete_series['loan_count'].fillna(0)\n",
    "    \n",
    "    return complete_series\n",
    "\n",
    "# Create quarterly time series\n",
    "if not hmda_filtered.empty:\n",
    "    quarterly_series = create_quarterly_series(hmda_filtered)\n",
    "    \n",
    "    print(\"Quarterly Time Series:\")\n",
    "    print(f\"Time range: {quarterly_series['date'].min()} to {quarterly_series['date'].max()}\")\n",
    "    print(f\"Total quarters: {len(quarterly_series)}\")\n",
    "    \n",
    "    display(quarterly_series.head(10))\n",
    "    \n",
    "    # Summary statistics\n",
    "    print(\"\\nQuarterly Volume Summary:\")\n",
    "    print(f\"Total volume: ${quarterly_series['total_loan_volume'].sum():,.0f}\")\n",
    "    print(f\"Average quarterly volume: ${quarterly_series['total_loan_volume'].mean():,.0f}\")\n",
    "    print(f\"Max quarterly volume: ${quarterly_series['total_loan_volume'].max():,.0f}\")\n",
    "    print(f\"Min quarterly volume: ${quarterly_series['total_loan_volume'].min():,.0f}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7. Save Processed Data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Save processed data\n",
    "if not quarterly_series.empty:\n",
    "    output_path = Path(\"../data/processed/quarterly_mortgage_volume.csv\")\n",
    "    output_path.parent.mkdir(parents=True, exist_ok=True)\n",
    "    \n",
    "    quarterly_series.to_csv(output_path, index=False)\n",
    "    print(f\"\\nProcessed data saved to: {output_path}\")\n",
    "    \n",
    "    # Also save the filtered HMDA data for future use\n",
    "    filtered_path = Path(\"../data/processed/filtered_hmda_data.csv\")\n",
    "    hmda_filtered.to_csv(filtered_path, index=False)\n",
    "    print(f\"Filtered HMDA data saved to: {filtered_path}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 8. Quick Visualization"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if not quarterly_series.empty:\n",
    "    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))\n",
    "    \n",
    "    # Plot 1: Total loan volume\n",
    "    ax1.plot(quarterly_series['date'], quarterly_series['total_loan_volume'] / 1e6, \n",
    "             linewidth=2, marker='o', markersize=4)\n",
    "    ax1.set_title(f'Quarterly Mortgage Origination Volume\\n{config[\"data\"][\"target_geography\"]}', \n",
    "                 fontsize=14, fontweight='bold')\n",
    "    ax1.set_ylabel('Volume (Millions $)')\n",
    "    ax1.grid(True, alpha=0.3)\n",
    "    \n",
    "    # Plot 2: Loan count\n",
    "    ax2.plot(quarterly_series['date'], quarterly_series['loan_count'], \n",
    "             linewidth=2, marker='o', markersize=4, color='orange')\n",
    "    ax2.set_title('Quarterly Loan Count', fontsize=14, fontweight='bold')\n",
    "    ax2.set_ylabel('Number of Loans')\n",
    "    ax2.set_xlabel('Date')\n",
    "    ax2.grid(True, alpha=0.3)\n",
    "    \n",
    "    plt.tight_layout()\n",
    "    plt.show()\n",
    "    \n",
    "    # Save the plot\n",
    "    fig.savefig('../outputs/data_wrangling_timeseries.png', dpi=300, bbox_inches='tight')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Summary\n",
    "\n",
    "In this notebook we:\n",
    "1. Loaded and combined multiple years of HMDA data\n",
    "2. Filtered for originated loans in our target geography\n",
    "3. Performed data quality checks\n",
    "4. Aggregated to quarterly time series\n",
    "5. Saved processed data for analysis\n",
    "6. Created initial visualizations\n",
    "\n",
    "The data is now ready for exploratory analysis in the next notebook."
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}